In [1]:
import numpy as np
from pymatgen.ext.matproj import MPRester
from pymatgen.core import Structure
from fireworks import LaunchPad
import os
from atomate.vasp.powerups import add_modify_incar, add_modify_potcar, add_tags
from ph3pywf.workflows.core import wf_phono3py
from ph3pywf.workflows.core import wf_disp_from_optimized

successfully loaded your custom FW_config.yaml!


/opt/conda/lib/python3.8/site-packages/maggma/utils.py:20: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


# Full phono3py workflow
see below

In [ ]:
### Running conditions ##############################################
material_id = "mp-2574" # Material id to be searched on MP
supercell_size = (2,2,2)

### Get working dir #################################################
working_dir = os.getcwd()

### Get materials structure #########################################
## If no local structure file, get from MP and save unitcell
## structure as "struct_unitcell.json"
if "struct_unitcell.json" in os.listdir(working_dir):
    print("Found local structure file")
    ## Read saved local structure file
    struct_unitcell = Structure.from_file("struct_unitcell.json")

else:
    print("Getting structure from MaterialsProject")
    # api_key = ''
    with MPRester() as mpr:
        struct_unitcell = mpr.get_structure_by_material_id(material_id)

    ## Save a local structure file to avoid accessing MP every time
    with open("struct_unitcell.json", "w") as fh:
        fh.write(struct_unitcell.to_json())

### Create the workflow #############################################
c = {"supercell_size": supercell_size, 
     "cutoff_pair_distance": 5.0, 
    }

print("Creating workflow...")
workflow = wf_phono3py(structure=struct_unitcell, 
                       c=c,
                      )
print("Created workflow")

### Add Powerups ####################################################
modify_incar_params = {'incar_update': {'ENCUT': 520,
                                        'ALGO': 'Normal'}}

modify_potcar_params = {'potcar_symbols': {
    'Zr': 'Zr_sv'}, 'functional': 'PBE_54'}
# modify incar
workflow = add_modify_incar(workflow, modify_incar_params=modify_incar_params)

# modify potcar
workflow = add_modify_potcar(workflow, modify_potcar_params=modify_potcar_params)

# # add tags
# workflow = add_tags(workflow, ["test"])


### Create the launchpad and add our workflow #######################
print("Sending to LaunchPad...")
launchpad = LaunchPad.auto_load()
launchpad.add_wf(workflow)
print("Sent to LaunchPad")

# Testing workflow using existing optimized structure
see below

In [3]:
### Running conditions ##############################################
material_id = "mp-2574" # Material id to be searched on MP
supercell_size = (2,2,2)

### Get working dir #################################################
working_dir = os.getcwd()
# db_file = os.path.join(working_dir, "db.json")

### Get materials structure #########################################
## If no local structure file, get from MP and save unitcell
## structure as "struct_unitcell.json"
if "struct_unitcell.json" in os.listdir(working_dir):
    print("Found local structure file")
    ## Read saved local structure file
    struct_unitcell = Structure.from_file("struct_unitcell.json")

else:
    print("Getting structure from MaterialsProject")
    # api_key = ''
    with MPRester() as mpr:
        struct_unitcell = mpr.get_structure_by_material_id(material_id)

    ## Save a local structure file to avoid accessing MP every time
    with open("struct_unitcell.json", "w") as fh:
        fh.write(struct_unitcell.to_json())

### Create the workflow #############################################
print("Creating workflow...")
workflow = wf_disp_from_optimized(structure=struct_unitcell, 
                                  supercell_size=supercell_size, 
                                 )
print("Created workflow")

### Create the launchpad and add our workflow #######################
print("Sending to LaunchPad...")
launchpad = LaunchPad.from_file("/home/jovyan/atomate/config/my_launchpad.yaml")
launchpad.add_wf(workflow)
print("Sent to LaunchPad")

Found local structure file
Creating workflow...
Created workflow
Sending to LaunchPad...
2021-05-24 03:53:23,193 INFO Added a workflow. id_map: {-2: 694}
Sent to LaunchPad
